# Predict Bike Rental

In this notebook we will be loading an external dataset into pytorch and train a neural network with it.

In [ ]:
import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import numpy as np
import time
import os


import matplotlib.pyplot as plt
%matplotlib inline

args = {
    'epoch_num': 200,     
    'lr': 5e-5,           
    'weight_decay': 5e-4, 
    'num_workers': 3,     
    'batch_size': 20,     
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

## Loading Dataset 

### Download the dataset

ref: https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset



In [ ]:
# Commented in order to avoid download the dataset again.
#! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
#! unzip Bike-Sharing-Dataset.zip  

### Checking data

In [ ]:
import pandas as pd
df = pd.read_csv('hour.csv')
print(len(df))
df.head()

### Spliting dataset into train and test

`torch.randperm` selects a random permutation of integers from 0 to len(df) - 1.

Doc: https://pytorch.org/docs/stable/torch.html#torch.randperm

In [ ]:

torch.manual_seed(1)
indices = torch.randperm(len(df)).tolist()

train_size = int(0.8*len(df))
df_train = df.iloc[indices[:train_size]]
df_test  = df.iloc[indices[train_size:]]

print(len(df_train), len(df_test))
display(df_test.head())

df_train.to_csv('bike_train.csv',index=False)
df_test.to_csv('bike_test.csv',index=False)
!ls

### Creating Dataset Class

`torch.util.data.Dataset` - Allows us to implement our own dataset.

Overwrite methods?
* ```__init__(self)```: Define the list of data from your dataset
* ```__getitem__(self, idx)```: Load a data, transforms and returns a tuple `(data, result)`.
* ```__len__(self)```: Returns the dataset lenght

ref: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html


In [ ]:
class BikeRental(Dataset):
  def __init__(self, csv_path, scaler_feat=None, scaler_label=None):
  
    self.dados = pd.read_csv(csv_path).to_numpy()
    
  def __getitem__(self, idx):
    
    sample = self.dados[idx][2:14]
    label  = self.dados[idx][-1:]

    sample = torch.from_numpy(sample.astype(np.float32))
    label  = torch.from_numpy(label.astype(np.float32))
    
    return sample, label
    
  def __len__(self):
    return len(self.dados)

In [ ]:
dataset = BikeRental('bike_train.csv')
dado, rotulo = dataset[0]
print(rotulo)
print(dado)

### Defining train and test datasets

In [ ]:
train_set = BikeRental('bike_train.csv')
test_set  = BikeRental('bike_test.csv')

print('Train: ' + str(len(train_set)))
print('Test: ' + str(len(test_set)))

## Dataloader


In [ ]:
train_loader = DataLoader(train_set,
                          args['batch_size'],
                          # num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         args['batch_size'],
                         # num_workers=args['num_workers'],
                         shuffle=False)

# TODO: find a solution for the problem with num_workers and Jupyter

O objeto retornado é um **iterador**, podendo ser utilizado para iterar em loops mas não suportando indexação.

In [ ]:
for batch in test_loader:
  dado, rotulo = batch
  print('## Batch Dimension ##')
  print(dado.size(), rotulo.size())
  break

## Creating 2 hidden layer MLP

In [ ]:
class MLP(nn.Module):
  
  def __init__(self, input_size, hidden_size, out_size):
    super(MLP, self).__init__()
    
    self.features = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
    )
    
    self.classifier = nn.Sequential(
        nn.Linear(hidden_size, out_size),
        nn.ReLU(),
    )

  def forward(self, X):
    
    hidden = self.features(X)
    output = self.classifier(hidden)
    
    return output

input_size  = train_set[0][0].size(0)
hidden_size = 128
out_size    = 1

net = MLP(input_size, hidden_size, out_size).to(args['device'])
print(net)

## Loss function and Optimizer

In [ ]:
criterion = nn.L1Loss().to(args['device'])

optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

## Training and Validation

In [ ]:
def train(train_loader, net, epoch):

  # Training mode
  net.train()
  
  start = time.time()
  
  epoch_loss  = []
  for batch in train_loader:
    
    dado, rotulo = batch
    
    # Cast do dado na GPU
    dado = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])
    
    # Forward
    ypred = net(dado)
    loss = criterion(ypred, rotulo)
    epoch_loss.append(loss.cpu().data)
    
    # Backpropagation
    loss.backward()
    optimizer.step()
   
  epoch_loss = np.asarray(epoch_loss)
  
  end = time.time()
  print('#################### Train ####################')
  print('Epoch %d, Loss: %.4f +/- %.4f, Time: %.2f' % (epoch, epoch_loss.mean(), epoch_loss.std(), end-start))
  
  return epoch_loss.mean()
    

In [ ]:
def validate(test_loader, net, epoch):

  # Evaluation mode
  net.eval()
  
  start = time.time()
  
  epoch_loss  = []
  
  with torch.no_grad(): 
    for batch in test_loader:

      dado, rotulo = batch

      # Cast do dado na GPU
      dado = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward
      ypred = net(dado)
      loss = criterion(ypred, rotulo)
      epoch_loss.append(loss.cpu().data)

  epoch_loss = np.asarray(epoch_loss)
  
  end = time.time()
  print('********** Validate **********')
  print('Epoch %d, Loss: %.4f +/- %.4f, Time: %.2f\n' % (epoch, epoch_loss.mean(), epoch_loss.std(), end-start))
  
  return epoch_loss.mean()
    

In [ ]:
train_losses, test_losses = [], []
for epoch in range(args['epoch_num']):
  
  # Train
  train_losses.append(train(train_loader, net, epoch))
  
  # Validate
  test_losses.append(validate(test_loader, net, epoch))

In [ ]:
Xtest = torch.stack([tup[0] for tup in test_set])
Xtest = Xtest.to(args['device'])

ytest = torch.stack([tup[1] for tup in test_set])
ypred = net(Xtest).cpu().data

data = torch.cat((ytest, ypred), axis=1)

df_results = pd.DataFrame(data, columns=['ypred', 'ytest'])
df_results.head(20)

# Gráfico de convergência

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(train_losses, label='Train')
plt.plot(test_losses, label='Test', linewidth=3, alpha=0.5)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Convergence', fontsize=16)
plt.legend()
plt.show()